In [183]:
import numpy as np

sample_np = np.random.rand(100, 2)
sample_np.shape

(100, 2)

In [184]:
from sympy import symbols


def polynomial():
    x1 = symbols('x1')
    x2 = symbols('x2')
    return 4*x1**2 + 5*x2**2 + 2*x1*x2 + 3*x1 - 6*x2

In [185]:
from sklearn.preprocessing import PolynomialFeatures
from sympy import Poly, lambdify


def polynomial_regression_gradient_descent(features: np.ndarray, poloinomial_func, learning_rate: float, epochs: int):
    # Визначаємо степінь полінома з символьного виразу
    ploinomial_rate = Poly(poloinomial_func).total_degree()

    # Готуємо ціль як значення полінома на даних
    x1, x2 = symbols('x1 x2')
    f = lambdify((x1, x2), poloinomial_func, modules='numpy')
    y = f(features[:, 0], features[:, 1]).astype(float).reshape(-1)  # (m,)

    # Узгоджено генеруємо поліноміальні ознаки у train
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(features)  # (m, p)
    m, p = X_poly.shape

    # Ключова правка: довжина theta дорівнює кількості колонок у X_poly
    theta = np.zeros(p, dtype=float)

    for _ in range(epochs):
        y_pred = X_poly @ theta          # (m,)
        error = y_pred - y               # (m,)
        gradient = (2.0 / m) * (X_poly.T @ error)  # (p,)
        theta -= learning_rate * gradient

    return theta

In [186]:
def polynomial_regression_SGD(features: np.ndarray, poloinomial_func, learning_rate: float, epochs: int, shuffle: bool = True):
    # 1) Цільові значення з символічного полінома
    x1, x2 = symbols('x1 x2')
    f = lambdify((x1, x2), poloinomial_func, modules='numpy')
    y = f(features[:, 0], features[:, 1]).astype(float).reshape(-1)  # (m,)

    # 2) Поліноміальні ознаки узгоджено з іншими функціями
    ploinomial_rate = Poly(poloinomial_func).total_degree()
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(features)  # (m, p)
    m, p = X_poly.shape

    # 3) Ініціалізація ваг
    theta = np.zeros(p, dtype=float)

    # 4) Класичний SGD: обхід по зразках
    for _ in range(epochs):
        if shuffle:
            perm = np.random.permutation(m)
            X_poly_epoch = X_poly[perm]
            y_epoch = y[perm]
        else:
            X_poly_epoch = X_poly
            y_epoch = y

        for i in range(m):
            xi = X_poly_epoch[i]          # (p,)
            yi = y_epoch[i]               # scalar
            y_pred_i = xi @ theta         # scalar
            error_i = y_pred_i - yi       # scalar
            grad_i = 2.0 * xi * error_i   # (p,)
            theta -= learning_rate * grad_i

    return theta


In [187]:
def polynomial_regression_rmsprop(features: np.ndarray, poloinomial_func, learning_rate: float, epochs: int, gamma = 0.9, epsilon = 1e-8):
    # 1) Цільові значення з символічного полінома
    x1, x2 = symbols('x1 x2')
    f = lambdify((x1, x2), poloinomial_func, modules='numpy')
    y = f(features[:, 0], features[:, 1]).astype(float).reshape(-1)  # (m,)

    # 2) Поліноміальні ознаки узгоджено з іншими функціями (include_bias=True)
    ploinomial_rate = Poly(poloinomial_func).total_degree()
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(features)  # (m, p)
    m, p = X_poly.shape

    # 3) Ініціалізація ваг та накопичувача квадратів градієнтів
    theta = np.zeros(p, dtype=float)
    eg2 = np.zeros(p, dtype=float)

    # 4) RMSProp із повним батчем
    for _ in range(epochs):
        y_pred = X_poly @ theta                 # (m,)
        error = y_pred - y                      # (m,)
        grad = (2.0 / m) * (X_poly.T @ error)   # (p,)

        eg2 = gamma * eg2 + (1.0 - gamma) * (grad ** 2)     # (p,)
        theta -= (learning_rate / (np.sqrt(eg2) + epsilon)) * grad

    return theta

In [188]:
def predict(X_input: np.ndarray, poloinomial_func, theta: np.ndarray):
    # Та сама логіка PolynomialFeatures, що і в train
    ploinomial_rate = Poly(poloinomial_func).total_degree()
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(X_input)  # Порядок і набір ознак детерміновані degree і n_features
    return X_poly @ theta



In [189]:
def actual_vals(X_input: np.ndarray, poloinomial_func):
    x1, x2 = symbols('x1 x2')
    polynomial = lambdify((x1, x2), poloinomial_func, modules='numpy')
    return polynomial(X_input[:, 0], X_input[:, 1])

In [190]:
test_np = np.random.rand(10, 2)
pol_func = polynomial()
pred_gd = predict(test_np, pol_func, polynomial_regression_gradient_descent(sample_np, pol_func, 0.1, 10000))
pred_gd

array([ 0.9312045 ,  1.18898098,  0.53067088,  4.38903224,  4.45750542,
        4.72301814, -0.99459076,  4.13752696, -0.79096251,  6.86920502])

In [191]:
pred_rmsprop = predict(test_np, pol_func, polynomial_regression_rmsprop(sample_np, pol_func, 0.02, 10000))
pred_rmsprop

array([ 0.94855695,  1.2224301 ,  0.54737012,  4.41723507,  4.4872999 ,
        4.75467221, -0.97633528,  4.16419854, -0.77211814,  6.93206886])

In [192]:
pred_sgd = predict(test_np, pol_func, polynomial_regression_SGD(sample_np, pol_func, 0.1, 10000))
pred_sgd

array([ 0.9321868 ,  1.18678361,  0.52966238,  4.39038299,  4.45772663,
        4.72342086, -0.99561382,  4.13906156, -0.79326384,  6.87639064])

In [193]:
actual = actual_vals(test_np, pol_func)
actual

array([ 0.9321868 ,  1.18678361,  0.52966238,  4.39038299,  4.45772663,
        4.72342086, -0.99561382,  4.13906156, -0.79326384,  6.87639064])